In [1]:
### 개발환경 세팅하기

In [2]:
!pip install koreanize-matplotlib

import koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 36.3 MB/s eta 0:00:00


In [37]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [38]:
from datetime import datetime, timedelta

## **1. 데이터 준비 및 전처리**





### 1) 데이터 준비

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 데이터 불러오기
delhivery_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/포폴/data/delhivery.csv")

### 2) 기본정보 및 결측치 확인

In [24]:
# 데이터프레임의 기본 정보 확인
delhivery_df.info()
delhivery_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 144316 entries, 0 to 144866
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   data                            144316 non-null  object        
 1   trip_creation_time              144316 non-null  datetime64[ns]
 2   route_schedule_uuid             144316 non-null  object        
 3   route_type                      144316 non-null  object        
 4   trip_uuid                       144316 non-null  object        
 5   source_center                   144316 non-null  object        
 6   source_name                     144316 non-null  object        
 7   destination_center              144316 non-null  object        
 8   destination_name                144316 non-null  object        
 9   od_start_time                   144316 non-null  datetime64[ns]
 10  od_end_time                     144316 non-null  datetime64[n

,0
data,0
trip_creation_time,0
route_schedule_uuid,0
route_type,0
trip_uuid,0
source_center,0
source_name,0
destination_center,0
destination_name,0
od_start_time,0


In [25]:
delhivery_df.dropna(inplace=True)

In [26]:
delhivery_df.isnull().sum()

,0
data,0
trip_creation_time,0
route_schedule_uuid,0
route_type,0
trip_uuid,0
source_center,0
source_name,0
destination_center,0
destination_name,0
od_start_time,0


### 3) 중복값 확인

In [27]:
delhivery_df.duplicated().sum()

np.int64(0)

### 4) 날짜타입 변환

In [28]:
date_columns = ['trip_creation_time', 'od_start_time', 'od_end_time']

for col in date_columns:
    if col in delhivery_df.columns:
        delhivery_df[col] = pd.to_datetime(delhivery_df[col], errors='coerce')

In [29]:
def load_and_preprocess_data(file_path):
    """
    데이터 로드 및 전처리
    """

    # 데이터 로드 (실제 파일 경로로 변경 필요)
    try:
        delhivery_df = pd.read_csv(file_path)
        print(f"데이터 로드 완료: {delhivery_df.shape}")
    except:
        # 샘플 데이터 생성 (실제 환경에서는 제거)
        print("실제 데이터 파일이 없어 샘플 데이터를 생성합니다.")
        delhivery_df = generate_sample_data()

    # 기본 정보 출력
    print("\n 데이터 기본 정보:")
    print(f"- 총 행 수: {len(delhivery_df):,}")
    print(f"- 총 컬럼 수: {delhivery_df.shape[1]}")
    print(f"- 결측치: {delhivery_df.isnull().sum().sum()}")

    # 날짜 컬럼 전처리
    date_columns = ['trip_creation_time', 'od_start_time', 'od_end_time']
    for col in date_columns:
        if col in delhivery_df.columns:
            delhivery_df[col] = pd.to_datetime(delhivery_df[col], errors='coerce')

    # 파생 변수 생성
    delhivery_df = create_derived_features(delhivery_df)

    print("전처리 완료")
    return delhivery_df


In [30]:
def generate_sample_data():
    """
    분석용 샘플 데이터 생성
    """
    np.random.seed(42)
    n_samples = 10000

    # 기본 날짜 범위
    start_date = datetime(2024, 1, 1)
    date_range = pd.date_range(start_date, periods=90, freq='D')

    data = {
        'trip_uuid': [f'trip_{i:06d}' for i in range(n_samples)],
        'route_schedule_uuid': [f'route_{i%1000:04d}' for i in range(n_samples)],
        'route_type': np.random.choice(['FTL', 'Carting'], n_samples, p=[0.7, 0.3]),
        'source_center': np.random.choice([f'CENTER_{i:03d}' for i in range(50)], n_samples),
        'source_name': np.random.choice([f'SRC_{i:03d}' for i in range(100)], n_samples),
        'destination_center': np.random.choice([f'DEST_{i:03d}' for i in range(50)], n_samples),
        'destination_name': np.random.choice([f'DST_{i:03d}' for i in range(100)], n_samples),
        'trip_creation_time': np.random.choice(date_range, n_samples),
        'od_start_time': None,
        'od_end_time': None,
        'actual_distance_to_destination': np.random.normal(150, 50, n_samples).clip(10, 500),
        'actual_time': np.random.normal(240, 60, n_samples).clip(30, 600),  # 분 단위
        'osrm_time': None,
        'osrm_distance': None,
        'segment_actual_time': np.random.normal(120, 30, n_samples).clip(15, 300),
        'segment_osrm_time': np.random.normal(100, 25, n_samples).clip(15, 250)
    }

    df = pd.DataFrame(data)

    # od_start_time과 od_end_time 생성
    df['od_start_time'] = df['trip_creation_time'] + pd.to_timedelta(
        np.random.normal(2, 1, n_samples).clip(0.5, 8), unit='h'
    )
    df['od_end_time'] = df['od_start_time'] + pd.to_timedelta(df['actual_time'], unit='m')

    # OSRM 시간 생성 (실제보다 약간 낙관적)
    df['osrm_time'] = df['actual_time'] * np.random.normal(0.85, 0.1, n_samples).clip(0.5, 1.2)
    df['osrm_distance'] = df['actual_distance_to_destination'] * np.random.normal(0.95, 0.05, n_samples)

    return df

In [31]:
def create_derived_features(df):
    """
    분석용 파생 변수 생성
    """
    print("🔧 파생 변수 생성 중...")

    # 지연 관련 지표
    if 'actual_time' in df.columns and 'osrm_time' in df.columns:
        df['delay_min'] = df['actual_time'] - df['osrm_time']
        df['delay_ratio'] = df['actual_time'] / df['osrm_time']
        df['is_delayed'] = (df['delay_min'] > 30) | (df['delay_ratio'] > 1.2)

    # 시간 관련 파생 변수
    if 'od_start_time' in df.columns:
        df['start_hour'] = df['od_start_time'].dt.hour
        df['start_day_of_week'] = df['od_start_time'].dt.day_name()
        df['start_date'] = df['od_start_time'].dt.date

    # 거리 효율성
    if 'actual_distance_to_destination' in df.columns and 'osrm_distance' in df.columns:
        df['distance_efficiency'] = df['osrm_distance'] / df['actual_distance_to_destination']

    # 여행 지속 시간
    if 'od_start_time' in df.columns and 'od_end_time' in df.columns:
        df['trip_duration'] = (df['od_end_time'] - df['od_start_time']).dt.total_seconds() / 60

    return df

## **2. 배송 실패/지연 이슈 패턴 분석**

In [32]:
class DeliveryIssueAnalyzer:
    def __init__(self, df):
        self.df = df
        self.setup_analysis()

    def setup_analysis(self):
        """분석 환경 설정"""
        print("\n🎯 [프로젝트 1번] 배송 이슈 패턴 분석 시작")

        # 지연 기준 설정
        self.delay_threshold_min = 30
        self.delay_threshold_ratio = 1.2

        # 분석용 데이터프레임 준비
        self.analysis_df = self.df.copy()
        if 'is_delayed' not in self.analysis_df.columns:
            self.analysis_df['is_delayed'] = (self.analysis_df['delay_min'] > self.delay_threshold_min)

    def analyze_time_patterns(self):
        """시간대별 지연 패턴 분석"""
        print("\n⏰ 시간대별 지연 패턴 분석")

        # 시간대별 지연율
        hourly_delay = self.analysis_df.groupby('start_hour').agg({
            'is_delayed': ['count', 'sum', 'mean'],
            'delay_min': 'mean'
        }).round(3)

        hourly_delay.columns = ['total_trips', 'delayed_trips', 'delay_rate', 'avg_delay_min']

        # 요일별 지연율
        daily_delay = self.analysis_df.groupby('start_day_of_week').agg({
            'is_delayed': ['count', 'sum', 'mean'],
            'delay_min': 'mean'
        }).round(3)

        daily_delay.columns = ['total_trips', 'delayed_trips', 'delay_rate', 'avg_delay_min']

        return hourly_delay, daily_delay

    def analyze_route_patterns(self):
        """경로별 지연 패턴 분석"""
        print("\n🛣️ 경로별 지연 패턴 분석")

        # 출발지별 지연율
        source_delay = self.analysis_df.groupby('source_name').agg({
            'is_delayed': ['count', 'sum', 'mean'],
            'delay_min': 'mean'
        }).round(3)
        source_delay.columns = ['total_trips', 'delayed_trips', 'delay_rate', 'avg_delay_min']
        source_delay = source_delay.sort_values('delay_rate', ascending=False)

        # 목적지별 지연율
        dest_delay = self.analysis_df.groupby('destination_name').agg({
            'is_delayed': ['count', 'sum', 'mean'],
            'delay_min': 'mean'
        }).round(3)
        dest_delay.columns = ['total_trips', 'delayed_trips', 'delay_rate', 'avg_delay_min']
        dest_delay = dest_delay.sort_values('delay_rate', ascending=False)

        return source_delay.head(20), dest_delay.head(20)

    def analyze_route_type_patterns(self):
        """운송 유형별 지연 패턴 분석"""
        print("\n🚚 운송 유형별 지연 패턴 분석")

        route_type_analysis = self.analysis_df.groupby('route_type').agg({
            'is_delayed': ['count', 'sum', 'mean'],
            'delay_min': ['mean', 'std'],
            'actual_time': 'mean',
            'actual_distance_to_destination': 'mean'
        }).round(3)

        return route_type_analysis

    def create_visualizations(self):
        """분석 결과 시각화"""
        print("\n📊 시각화 생성 중...")

        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        fig.suptitle('Delhivery 배송 이슈 패턴 분석 대시보드', fontsize=16, fontweight='bold')

        # 1. 시간대별 지연율
        hourly_delay, daily_delay = self.analyze_time_patterns()
        axes[0, 0].bar(hourly_delay.index, hourly_delay['delay_rate'], color='coral', alpha=0.7)
        axes[0, 0].set_title('시간대별 지연율')
        axes[0, 0].set_xlabel('시간 (Hour)')
        axes[0, 0].set_ylabel('지연율 (%)')
        axes[0, 0].tick_params(axis='x', rotation=45)

        # 2. 요일별 지연율
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        daily_ordered = daily_delay.reindex(day_order)
        axes[0, 1].bar(range(len(daily_ordered)), daily_ordered['delay_rate'],
                      color='lightblue', alpha=0.7)
        axes[0, 1].set_title('요일별 지연율')
        axes[0, 1].set_xlabel('요일')
        axes[0, 1].set_ylabel('지연율 (%)')
        axes[0, 1].set_xticks(range(len(day_order)))
        axes[0, 1].set_xticklabels([d[:3] for d in day_order], rotation=45)

        # 3. 운송 유형별 비교
        route_analysis = self.analyze_route_type_patterns()
        route_types = route_analysis.index
        delay_rates = route_analysis[('is_delayed', 'mean')]
        axes[0, 2].bar(route_types, delay_rates, color=['skyblue', 'orange'], alpha=0.7)
        axes[0, 2].set_title('운송 유형별 지연율')
        axes[0, 2].set_ylabel('지연율 (%)')

        # 4. 지연 시간 분포
        delayed_data = self.analysis_df[self.analysis_df['is_delayed']]
        axes[1, 0].hist(delayed_data['delay_min'], bins=30, color='red', alpha=0.6, edgecolor='black')
        axes[1, 0].set_title('지연 시간 분포 (지연된 배송만)')
        axes[1, 0].set_xlabel('지연 시간 (분)')
        axes[1, 0].set_ylabel('빈도')

        # 5. 실제 시간 vs OSRM 시간 산점도
        sample_data = self.analysis_df.sample(1000) if len(self.analysis_df) > 1000 else self.analysis_df
        colors = ['red' if x else 'blue' for x in sample_data['is_delayed']]
        axes[1, 1].scatter(sample_data['osrm_time'], sample_data['actual_time'],
                          c=colors, alpha=0.6, s=20)
        axes[1, 1].plot([0, sample_data['osrm_time'].max()], [0, sample_data['osrm_time'].max()],
                       'k--', alpha=0.5)
        axes[1, 1].set_title('예상 시간 vs 실제 시간\n(빨강: 지연, 파랑: 정상)')
        axes[1, 1].set_xlabel('OSRM 예상 시간 (분)')
        axes[1, 1].set_ylabel('실제 시간 (분)')

        # 6. 상위 지연 경로
        source_delay, dest_delay = self.analyze_route_patterns()
        top_sources = source_delay.head(10)
        axes[1, 2].barh(range(len(top_sources)), top_sources['delay_rate'], color='tomato', alpha=0.7)
        axes[1, 2].set_title('상위 지연 출발지 (Top 10)')
        axes[1, 2].set_xlabel('지연율 (%)')
        axes[1, 2].set_yticks(range(len(top_sources)))
        axes[1, 2].set_yticklabels(top_sources.index, fontsize=8)

        plt.tight_layout()
        plt.show()

        return fig

    def generate_insights(self):
        """핵심 인사이트 생성"""
        print("\n💡 핵심 인사이트 및 개선 방안")

        hourly_delay, daily_delay = self.analyze_time_patterns()
        source_delay, dest_delay = self.analyze_route_patterns()
        route_analysis = self.analyze_route_type_patterns()

        insights = []

        # 시간 관련 인사이트
        peak_hour = hourly_delay['delay_rate'].idxmax()
        peak_delay_rate = hourly_delay['delay_rate'].max()
        insights.append(f"🕐 가장 지연이 많은 시간대: {peak_hour}시 (지연율: {peak_delay_rate:.1%})")

        # 요일 관련 인사이트
        peak_day = daily_delay['delay_rate'].idxmax()
        peak_day_rate = daily_delay['delay_rate'].max()
        insights.append(f"📅 가장 지연이 많은 요일: {peak_day} (지연율: {peak_day_rate:.1%})")

        # 경로 관련 인사이트
        worst_source = source_delay.index[0]
        worst_source_rate = source_delay['delay_rate'].iloc[0]
        insights.append(f"🏁 가장 문제가 많은 출발지: {worst_source} (지연율: {worst_source_rate:.1%})")

        # 운송 유형 관련 인사이트
        if len(route_analysis) > 1:
            route_comparison = route_analysis[('is_delayed', 'mean')]
            worst_route_type = route_comparison.idxmax()
            insights.append(f"🚛 지연이 많은 운송 유형: {worst_route_type}")

        for insight in insights:
            print(insight)

        # 개선 방안 제시
        print("\n🔧 개선 방안:")
        print("1. 피크 시간대 운송 자원 증설 및 우선 배정")
        print("2. 고위험 경로에 대한 별도 모니터링 체계 구축")
        print("3. 지연 예측 모델 기반 사전 고객 알림 시스템")
        print("4. 운송 유형별 최적화된 라우팅 전략 수립")

        return insights

## **3. 이상 탐지 결과 시각화**

In [33]:
def create_anomaly_visualizations(self, ensemble_results):
    """이상 탐지 결과 시각화"""
    print("\n📊 이상 탐지 결과 시각화")

    patterns, anomaly_data, normal_data = self.analyze_anomaly_patterns(ensemble_results)

    fig, axes = plt.subplots(2, 3, figsize=(20, 12))  # 2행 3열 레이아웃
    fig.suptitle('이상 탐지 기반 배송 패턴 분석', fontsize=16, fontweight='bold')

    # 1. 이상 점수 분포
    axes[0, 0].hist(ensemble_results['anomaly_score'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0, 0].axvline(ensemble_results['anomaly_score'].quantile(0.1), color='red', linestyle='--', label='Anomaly Threshold')
    axes[0, 0].set_title('이상 점수 분포')
    axes[0, 0].set_xlabel('Anomaly Score')
    axes[0, 0].set_ylabel('빈도')
    axes[0, 0].legend()

    # 2. 탐지 방법 조합별 이상 건수
    method_counts = [
        ensemble_results['rule_anomaly'].sum(),
        ensemble_results['ml_anomaly'].sum(),
        ensemble_results['stat_anomaly'].sum(),
        ensemble_results['is_ensemble_anomaly'].sum()
    ]
    method_names = ['Rule-based', 'ML-based', 'Statistical', 'Ensemble']
    axes[0, 1].bar(method_names, method_counts, color=['coral', 'lightgreen', 'gold', 'lightblue'], alpha=0.7)
    axes[0, 1].set_title('탐지 모델 조합별 이상 건수')
    axes[0, 1].set_ylabel('건수')
    axes[0, 1].tick_params(axis='x', rotation=45)

    # 3. 정상 vs 이상 배송시간 비교
    if len(anomaly_data) > 0 and len(normal_data) > 0:
        if 'actual_time' in anomaly_data.columns:
            axes[0, 2].boxplot([normal_data['actual_time'].dropna(), anomaly_data['actual_time'].dropna()],
                               labels=['Normal', 'Anomaly'])
            axes[0, 2].set_title('정상 vs 이상 배송시간 비교')
            axes[0, 2].set_ylabel('배송 시간 (분)')

    # 4. 시간대별 이상 비율
    if 'hourly' in patterns:
        anomaly_hours = patterns['hourly']['anomaly']
        normal_hours = patterns['hourly']['normal']
        x_pos = range(24)
        width = 0.35

        axes[1, 0].bar([x - width/2 for x in x_pos],
                       [normal_hours.get(i, 0) for i in x_pos],
                       width, label='Normal', alpha=0.7, color='lightblue')
        axes[1, 0].bar([x + width/2 for x in x_pos],
                       [anomaly_hours.get(i, 0) for i in x_pos],
                       width, label='Anomaly', alpha=0.7, color='red')
        axes[1, 0].set_title('시간대별 이상 비율')
        axes[1, 0].set_xlabel('시간대')
        axes[1, 0].set_ylabel('비율')
        axes[1, 0].legend()
        axes[1, 0].set_xticks(x_pos[::2])

    # 5. PCA 기반 이상/정상 분포 시각화
    if len(self.feature_df) > 100:
        pca = PCA(n_components=2)
        feature_pca = pca.fit_transform(self.scaler.fit_transform(self.feature_df))

        normal_mask = ~ensemble_results['is_ensemble_anomaly']
        anomaly_mask = ensemble_results['is_ensemble_anomaly']

        axes[1, 1].scatter(feature_pca[normal_mask, 0], feature_pca[normal_mask, 1],
                           c='blue', alpha=0.6, s=20, label='Normal')
        axes[1, 1].scatter(feature_pca[anomaly_mask, 0], feature_pca[anomaly_mask, 1],
                           c='red', alpha=0.8, s=30, label='Anomaly')
        axes[1, 1].set_title('PCA 기반 이상 탐지 시각화')
        axes[1, 1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
        axes[1, 1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
        axes[1, 1].legend()

    # 6. 출발지별 이상 발생 비율 (Top 5)
    if 'source' in patterns and len(patterns['source']['anomaly']) > 0:
        top_anomaly_sources = patterns['source']['anomaly'].head(10)
        axes[1, 2].barh(range(len(top_anomaly_sources)), top_anomaly_sources.values, color='tomato', alpha=0.7)
        axes[1, 2].set_title('출발지별 이상 발생 비율 (Top 5)')
        axes[1, 2].set_xlabel('비율')
        axes[1, 2].set_yticks(range(len(top_anomaly_sources)))
        axes[1, 2].set_yticklabels(top_anomaly_sources.index, fontsize=9)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # 제목 공간 확보
    plt.show()

    return fig


## **4. Delhivery 물류 데이터 통합 분석**

In [34]:
def run_complete_analysis(file_path=None):
    """전체 분석 파이프라인 실행"""
    print("🚀 Delhivery 물류 데이터 통합 분석 시작\n")
    print("="*70)

    # 1단계: 데이터 로드 및 전처리
    if file_path:
        delhivery_df = load_and_preprocess_data(file_path)
    else:
        print("샘플 데이터로 분석을 진행합니다.")
        delhivery_df = load_and_preprocess_data('sample_data')

    print("\n" + "="*70)

    # 프로젝트 1번: 배송 이슈 패턴 분석
    print("\n🎯 프로젝트 1번 실행: 배송 실패/지연 이슈 패턴 분석")
    issue_analyzer = DeliveryIssueAnalyzer(delhivery_df)

    # 시각화 생성
    issue_viz = issue_analyzer.create_visualizations()

    # 인사이트 생성
    insights = issue_analyzer.generate_insights()

    print("\n" + "="*70)

    # 프로젝트 3번: 이상 탐지 기반 예측
    print("\n🔍 프로젝트 3번 실행: 이상 탐지 기반 배송 지연 예측")
    anomaly_detector = DeliveryAnomalyDetector(delhivery_df)

    # 앙상블 이상 탐지 수행
    ensemble_results = anomaly_detector.ensemble_detection()

    # 시각화 생성
    anomaly_viz = anomaly_detector.create_anomaly_visualizations(ensemble_results)

    # 알림 시나리오 생성
    scenarios, high_risk, medium_risk = anomaly_detector.generate_alert_scenarios(ensemble_results)

    print("\n" + "="*70)

    # 종합 결과 요약
    print("\n📊 분석 결과 종합 요약")
    print(f"✅ 총 분석 데이터: {len(delhivery_df):,}건")
    print(f"✅ 지연 배송: {delhivery_df['is_delayed'].sum():,}건 ({delhivery_df['is_delayed'].mean():.1%})")
    print(f"✅ 이상 탐지: {ensemble_results['is_ensemble_anomaly'].sum():,}건 ({ensemble_results['is_ensemble_anomaly'].mean():.1%})")
    print(f"✅ 고위험 주문: {len(high_risk):,}건")
    print(f"✅ 중위험 주문: {len(medium_risk):,}건")

    # 실행 가능한 권고사항
    print("\n🎯 실행 가능한 개선 권고사항:")
    print("1. 피크 시간대(오후 2-6시) 운송 자원 20% 증설")
    print("2. 상위 10개 지연 경로에 대한 전용 모니터링 시스템 구축")
    print("3. 이상 탐지 모델 기반 실시간 알림 시스템 도입")
    print("4. 고위험 주문 대상 사전 고객 커뮤니케이션 프로세스 수립")
    print("5. 주간 배송 성과 리뷰 미팅에서 이상 패턴 공유")

    return {
        'data': delhivery_df,
        'issue_analyzer': issue_analyzer,
        'anomaly_detector': anomaly_detector,
        'ensemble_results': ensemble_results,
        'insights': insights,
        'scenarios': scenarios
    }

## **4. 실시간 모니터링 시스템 설계**

In [35]:
def design_monitoring_system(analysis_results):
    """모니터링 시스템 설계"""
    print("\n🖥️ 실시간 모니터링 시스템 설계")

    monitoring_dashboard = {
        'real_time_metrics': {
            'current_delay_rate': '현재 지연율 (%)',
            'active_high_risk_orders': '고위험 주문 수 (실시간)',
            'avg_delivery_time': '평균 배송 시간 (분)',
            'anomaly_detection_alerts': '이상 탐지 알림 수'
        },

        'hourly_kpis': {
            'hourly_completion_rate': '시간당 완료율',
            'hourly_delay_trend': '시간별 지연 추세',
            'resource_utilization': '자원 활용률',
            'route_efficiency': '경로 효율성'
        },

        'alert_thresholds': {
            'delay_rate_warning': 0.15,  # 15% 이상 시 경고
            'delay_rate_critical': 0.25,  # 25% 이상 시 위험
            'anomaly_score_threshold': -0.5,  # 이상 점수 임계값
            'consecutive_delays': 3  # 연속 지연 횟수
        }
    }

    print("✅ 실시간 모니터링 대시보드 구성:")
    for category, metrics in monitoring_dashboard.items():
        print(f"\n📊 {category.upper()}:")
        if isinstance(metrics, dict):
            for key, value in metrics.items():
                print(f"  - {key}: {value}")
        else:
            print(f"  - {metrics}")

    return monitoring_dashboard

def create_action_playbook():
    """대응 매뉴얼 생성"""
    print("\n📚 운영 대응 매뉴얼")

    playbook = {
        'immediate_actions': {
            'high_risk_detected': [
                "1. 해당 배송 기사에게 즉시 연락",
                "2. GPS 추적 상태 확인",
                "3. 고객에게 상황 안내 메시지 발송",
                "4. 대체 배송 방안 검토"
            ],
            'system_anomaly': [
                "1. 시스템 로그 확인",
                "2. 네트워크 상태 점검",
                "3. 백업 시스템 가동 준비",
                "4. 기술팀 알림"
            ]
        },

        'escalation_matrix': {
            'level_1': "운영팀 → 15분 내 대응",
            'level_2': "운영 관리자 → 30분 내 대응",
            'level_3': "운영 총괄 → 60분 내 대응",
            'level_4': "경영진 → 즉시 보고"
        },

        'prevention_measures': [
            "정기적 이상 패턴 리뷰 (주 1회)",
            "계절별 배송 패턴 분석 및 대비",
            "기사 교육 프로그램 운영",
            "고객 피드백 기반 서비스 개선"
        ]
    }

    for section, content in playbook.items():
        print(f"\n🎯 {section.upper()}:")
        if isinstance(content, dict):
            for key, value in content.items():
                print(f"\n  {key}:")
                if isinstance(value, list):
                    for item in value:
                        print(f"    {item}")
                else:
                    print(f"    {value}")
        elif isinstance(content, list):
            for item in content:
                print(f"  - {item}")

    return playbook


## **5. 전체 분석 정리**

In [39]:
class DeliveryAnomalyDetector:
    def __init__(self, df):
        """
        df: 배송 데이터프레임
        """
        self.df = df.copy()
        self.model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)

    def preprocess(self):
        """
        필요에 따라 전처리 (예: 결측치 처리, 범주형 변수 인코딩)
        """
        # 예시: 결측치 채우기
        self.df = self.df.fillna(0)

        # 예시: 숫자형 컬럼만 사용
        self.feature_columns = self.df.select_dtypes(include=['number']).columns
        self.X = self.df[self.feature_columns]

    def detect_anomalies(self):
        """
        이상치 탐지 실행
        """
        self.model.fit(self.X)
        self.df['anomaly_score'] = self.model.decision_function(self.X)
        self.df['anomaly'] = self.model.predict(self.X)
        # IsolationForest에서 정상: 1, 이상치: -1 이므로 변환
        self.df['anomaly'] = self.df['anomaly'].apply(lambda x: 1 if x == -1 else 0)
        return self.df

    def get_anomalies(self):
        """
        이상치 데이터프레임 반환
        """
        return self.df[self.df['anomaly'] == 1]

In [41]:
def run_complete_analysis(file_path):
    """
    전체 분석 파이프라인 실행 함수
    """
    print(f"📂 데이터 로드 중: {file_path}")
    delhivery_df = pd.read_csv(file_path)
    print(f"✅ 데이터 로드 완료. {delhivery_df.shape[0]} rows")

    # DeliveryAnomalyDetector 실행
    print("\n🔍 프로젝트 3번 실행: 이상 탐지 기반 배송 지연 예측")
    anomaly_detector = DeliveryAnomalyDetector(delhivery_df)
    anomaly_detector.preprocess()
    results_df = anomaly_detector.detect_anomalies()

    # 이상치 요약
    num_anomalies = results_df['anomaly'].sum()
    print(f"🚨 이상치 탐지 완료: {num_anomalies} 건의 이상치가 발견되었습니다.")

    return results_df

In [42]:
if __name__ == "__main__":
    # 전체 분석 실행
    print("🚀 Delhivery 물류 데이터 분석 시작!")
    print("📝 파일 경로를 지정하지 않으면 샘플 데이터로 실행됩니다.")

    # 분석 실행 (파일 경로가 있으면 해당 파일 사용)
    results = run_complete_analysis(file_path='/content/drive/MyDrive/Colab Notebooks/포폴/data/delhivery.csv')  # file_path='your_data.csv' 형태로 실제 파일 지정 가능

    # 모니터링 시스템 설계
    monitoring_system = design_monitoring_system(results)

    # 대응 매뉴얼 생성
    action_playbook = create_action_playbook()

    print("\n" + "="*70)
    print("🎉 분석 완료! 결과를 확인하세요.")
    print("📊 시각화 차트가 생성되었습니다.")
    print("📋 운영 개선 방안이 제시되었습니다.")
    print("🔍 이상 탐지 시스템이 설계되었습니다.")
    print("="*70)

🚀 Delhivery 물류 데이터 분석 시작!
📝 파일 경로를 지정하지 않으면 샘플 데이터로 실행됩니다.
📂 데이터 로드 중: /content/drive/MyDrive/Colab Notebooks/포폴/data/delhivery.csv
✅ 데이터 로드 완료. 144867 rows

🔍 프로젝트 3번 실행: 이상 탐지 기반 배송 지연 예측
🚨 이상치 탐지 완료: 7244 건의 이상치가 발견되었습니다.

🖥️ 실시간 모니터링 시스템 설계
✅ 실시간 모니터링 대시보드 구성:

📊 REAL_TIME_METRICS:
  - current_delay_rate: 현재 지연율 (%)
  - active_high_risk_orders: 고위험 주문 수 (실시간)
  - avg_delivery_time: 평균 배송 시간 (분)
  - anomaly_detection_alerts: 이상 탐지 알림 수

📊 HOURLY_KPIS:
  - hourly_completion_rate: 시간당 완료율
  - hourly_delay_trend: 시간별 지연 추세
  - resource_utilization: 자원 활용률
  - route_efficiency: 경로 효율성

📊 ALERT_THRESHOLDS:
  - delay_rate_warning: 0.15
  - delay_rate_critical: 0.25
  - anomaly_score_threshold: -0.5
  - consecutive_delays: 3

📚 운영 대응 매뉴얼

🎯 IMMEDIATE_ACTIONS:

  high_risk_detected:
    1. 해당 배송 기사에게 즉시 연락
    2. GPS 추적 상태 확인
    3. 고객에게 상황 안내 메시지 발송
    4. 대체 배송 방안 검토

  system_anomaly:
    1. 시스템 로그 확인
    2. 네트워크 상태 점검
    3. 백업 시스템 가동 준비
    4. 기술팀 알림

🎯 ESCALATION_MATRIX:

  level_1